## Convert bird observations registered in artsobeservasjoner.no into a ebird.org record format

In [1]:
import pandas as pd
# import geopandas as gpd
from shapely.geometry import Point
import pyproj
# import geodatasets

ao_source = "minefunn_tom2019.xlsx"
ebird_template = "ebird_record_format_template.csv"

In [2]:
ao_df = pd.read_excel(ao_source, skiprows=2)
print(list(ao_df.columns))
ao_df[['Superlokalitet', 'Lokalitetsnavn','Østkoordinat', 'Nordkoordinat', 'Nøyaktighet', 'Originale koordinater',]].head(2)

['Id', 'Taksonsorteringsrekkefølge', 'Valideringsstatus', 'Rødlistekategori', 'Artsnavn', 'Vitenskapelig navn', 'Autor', 'Skjermet funn', 'Antall', 'Enhet', 'Alder', 'Kjønn', 'Aktivitet', 'Metode', 'Superlokalitet', 'Lokalitetsnavn', 'Østkoordinat', 'Nordkoordinat', 'Nøyaktighet', 'Originale koordinater', 'Fylke', 'Kommune', 'Fylke.1', 'IKKE I NORGE', 'Ekstern id', 'Startdato', 'Stattidspunkt', 'Sluttdato', 'Sluttidspunkt', 'Kommentar', 'Ikke gjenfunnet', 'Usikker artsbestemmelse', 'Uspontan', 'Natursystem', 'Natursystem beskrivelse', 'Livsmedium', 'Vitenskapelig livsmediumnavn', 'Art som livsmedium, beskrivelse', 'Livsmedium.1', 'Livsmediumbeskrivelse', 'Min. dybde', 'Maks. dybde', 'Høyde min', 'Høyde maks', 'Offentlig samling', 'Privat samling', 'Samlingsnummer', 'Samlingsbeskrivelse', 'Artsbestemt av', 'Bestemmelsesdato', 'Bekrefter', 'Bekreftelsesdato', 'Redigeringsansvarlig', 'Rapportør', 'Observatører', 'Prosjekt']


,Superlokalitet,Lokalitetsnavn,Østkoordinat,Nordkoordinat,Nøyaktighet,Originale koordinater
0,Runde,Goksøyr,325230,6924160,200,"Ø16359, N6954796 Sone 33 (±200m) UTM(WGS 84)"
1,Runde,"Lundeura, Runde",324316,6923443,0,"Ø15379, N6954165 Sone 33 (±0m) UTM(WGS 84)"


In [3]:
# Play with coordinates
easting = 16359
northing = 6954796
zone = 33
p = pyproj.Proj(proj='utm', zone=zone, datum='WGS84')
lon, lat = p(easting, northing, inverse=True)


# gdf = gpd.GeoDataFrame(data, crs='EPSG:4326')
# gdf.plot()

In [4]:
lon,lat

(5.616887295628967, 62.407573133496484)

In [5]:
ebird_headers = list(pd.read_csv(ebird_template, sep=";").columns)
# Add these headers to the ao list before we populate them
for ebird_header in ebird_headers:
    ao_df["ebird-"+ebird_header]= ""
ebird_headers

['Common Name',
 'Genus',
 'Species',
 'Number',
 'Species Comments',
 'Location Name',
 'Latitude',
 'Longitude',
 'Date',
 'Start Time',
 'State/Province',
 'Country Code',
 'Protocol',
 'Number of Observers',
 'Duration',
 'All observations reported?',
 'Effort Distance Miles',
 'Effort area acres',
 'Submission Comments']

In [7]:
# For each column we want to use, make the conversion here, quite explicitly
def latlong_from_orig(orig_text:str):
    splits = orig_text.split(" ")
    return splits

    p = pyproj.Proj(proj='utm', zone=zone, datum='WGS84')
    lon, lat = p(easting, northing, inverse=True)
# Kanskje gå over til østkoordinat, vestkoordinat igjen? Hva vet jeg om definisjonen her?
latlong_from_orig("Ø16359, N6954796 Sone 33 (±200m) UTM(WGS 84)")

['Ø16359,', 'N6954796', 'Sone', '33', '(±200m)', 'UTM(WGS', '84)']

In [ ]:

c = 'Common Name'
ao_df["ebird-"+c] = ao_df["Vitenskapelig navn"]

c = 'Number'
ao_df["ebird-"+c] = ao_df["Antall"]

c = 'Species Comments'
ao_df["ebird-"+c] = ao_df["Aktivitet"]

c =  'Date'
#TODO


# UTF to lat long
orig_koord = ""
for i, row in ao_df.copy().iterrows():
    orig_koord = row["Originale koordinater"]

c = 'Latitude'
#TODO

c = 'Longitude'
#TODO


orig_koord